# 线性回归的简洁实现

通过使用深度学习框架来简洁地实现
线性回归模型
生成数据集

In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

调用框架中现有的API来读取数据

In [28]:
def load_array(data_arrays, batch_size, is_train=True):  
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))

[tensor([[ 0.8445, -0.0765],
         [-2.9084, -0.2805],
         [-1.7490, -1.4440],
         [-0.6461,  0.1498],
         [ 0.9048,  1.1762],
         [-0.6307, -0.5761],
         [ 1.0622, -0.5502],
         [ 0.0221, -0.2559],
         [ 0.0337,  0.5994],
         [-1.5772,  1.0563]]),
 tensor([[ 6.1424],
         [-0.6483],
         [ 5.6156],
         [ 2.4149],
         [ 2.0113],
         [ 4.9035],
         [ 8.1737],
         [ 5.1305],
         [ 2.2353],
         [-2.5438]])]

使用框架的预定义好的层

In [29]:
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

初始化模型参数

In [35]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

计算均方误差使用的是`MSELoss`类，也称为平方$L_2$范数

In [36]:
loss = nn.MSELoss()

实例化一个`SGD`实例

In [37]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

训练过程代码与我们从零开始实现时所做的非常相似

In [38]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000239
epoch 2, loss 0.000100
epoch 3, loss 0.000100


比较生成数据集的真实参数和通过有限数据训练获得的模型参数

In [39]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([ 0.0007, -0.0005])
b的估计误差： tensor([-0.0006])
